<a href="https://colab.research.google.com/github/abuwildanm/food-recognition/blob/master/Food_Recognition_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indonesian Food Recognition

## Check GPU device

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Cloning and Building Darknet

The following cells will clone darknet from AlexeyAB's famous repository, adjust the Makefile to enable OPENCV and GPU for darknet and then build darknet.

In [ ]:
# Clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15043, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 15043 (delta 14), reused 19 (delta 8), pack-reused 15002
Receiving objects: 100% (15043/15043), 13.47 MiB | 15.43 MiB/s, done.
Resolving deltas: 100% (10213/10213), done.


In [ ]:
# Change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
# Make darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

## Download pretrained YOLOv3 weights

YOLOv3 has been trained already on the coco dataset which has 80 classes that it can predict. We will grab these pretrained weights so that we can run YOLOv3 on these pretrained classes and get detections.

In [ ]:
%cd /content/

/content


In [ ]:
# Download yolov3 pretrained weights
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://pjreddie.com/media/files/yolov3-tiny.weights
# Make yolov3 directory
!mkdir -p ./models/yolov3/
# Copy yolov3 weights to yolov3 directory
!cp {yolov3.weights,yolov3-tiny.weights} ./models/yolov3/
# Create symbolic link for yolov3 directory
!ln -s /content/models/yolov3 /yolov3

--2021-05-16 20:06:13--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  71.9MB/s    in 3.5s    

2021-05-16 20:06:17 (67.4 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2021-05-16 20:06:17--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  47.7MB/s    in 0.7s    

2021-05-16 20:06:18 (47.7 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [ ]:
# Define helper functions
def imshow(path):
    import matplotlib.pyplot as plt
    %matplotlib inline
    
    image = plt.imread(path)
    
    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(image)
    plt.show()

# Use this to upload files
def upload():
    from google.colab import files
    uploaded = files.upload() 
    for name, data in uploaded.items():
        with open(name, 'wb') as f:
            f.write(data)
            print ('saved file', name)

# Use this to download a file  
def download(path):
    from google.colab import files
    files.download(path)

## Run Object Detection with Darknet and YOLOv3

Darknet is now built and ready to run detections using YOLOv3 in the cloud! We can find out which sorts of classes the pretrained YOLOv3 weights can detect by clicking here. [COCO CLASSES](http://cocodataset.org/#explore)

The object detector can be run using the following command
```bash
!./darknet detect <path to config> <path to weights> <path to image>
```
Darknet comes with a few images already installed in the darknet/data/ folder.

**Note:** After running detections OpenCV can't open the image instantly in the cloud so we must run: 
```bash
imshow('predictions.jpg')
```
This will output the image with the detections shown. The most recent detections are always saved to 'predictions.jpg'

In [ ]:
%cd darknet

/content/darknet


In [ ]:
# Run darknet detection using yolov3 weights
!./darknet detect cfg/yolov3.cfg /yolov3/yolov3.weights data/person.jpg

CUDA status Error: file: ./src/dark_cuda.c : () : line: 39 : build time: May 16 2021 - 20:05:13 

 CUDA Error: no CUDA-capable device is detected
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
darknet: ./src/utils.c:331: error: Assertion `0' failed.


In [ ]:
# Result image from yolov3 weights
imshow('predictions.jpg')

FileNotFoundError: ignored

In [ ]:
# Run darknet detection using yolov3-tiny weights
!./darknet detect cfg/yolov3-tiny.cfg /yolov3/yolov3-tiny.weights data/person.jpg

In [ ]:
# Result image from yolov3-tiny weights
imshow('predictions.jpg')

## Download dataset from Google Cloud Storage

In [ ]:
%cd /content/

In [ ]:
# Login to our GCP account
!gcloud auth login

In [ ]:
# Set project using our project ID
!gcloud config set project food-nutrition-312416

In [ ]:
# Check the number of file in bucket
!gsutil ls -lR gs://food-nutrition/dataset | tail -n 1

In [ ]:
# Download dataset from our bucket
# !mkdir ./dataset
!gsutil -m cp -r gs://food-nutrition/dataset .

## View the dataset

In [ ]:
%cd /content/

In [ ]:
import os
from glob import glob

with open('/content/dataset/predefined-classes.txt') as f:
    categories = f.read().splitlines()

print('Number of train images:')
for category in categories:
    print('{}: {}'.format(category, len(glob('dataset/train/{}/*.jpg'.format(category)))))

print('='*40)
print('Number of train annotation:')
for category in categories:
    # -1 because doesn't include classes.txt
    print('{}: {}'.format(category, len(glob('dataset/train/{}/*.txt'.format(category)))-1))

print('='*40)
print('Number of validation images:')
for category in categories:
    print('{}: {}'.format(category, len(glob('dataset/validation/{}/*.jpg'.format(category)))))

print('='*40)
print('Number of validation annotation:')
for category in categories:
    # -1 because doesn't include classes.txt
    print('{}: {}'.format(category, len(glob('dataset/validation/{}/*.txt'.format(category)))-1))

In [ ]:
import matplotlib.pyplot as plt

def show_images(category, data_type='train', size=10, show_bbox=False):
    # Path of all image in category directory
    img_path = glob('dataset/{}/{}/*.jpg'.format(data_type, category))
    # Path of all annotation in category directory
    annotation_path = glob('dataset/{}/{}/*.txt'.format(data_type, category))
    # Shuffle the path
    # np.random.shuffle(img_path)
    # Plot images
    nrows = 2
    ncols = size / nrows
    fig = plt.figure(figsize=(15, 6))
    for idx in range(size):
        ax = fig.add_subplot(nrows, ncols, idx+1, xticks=[], yticks=[])
        image = plt.imread(img_path[idx])
        plt.title(os.path.basename(img_path[idx]))
        plt.imshow(image)

In [ ]:
coba = glob('dataset/train/omelette/*.txt')
ff = open(coba[19], 'r')
isi = ff.read().splitlines()
ff.close()

In [ ]:
show_images('omelette')